In [0]:
%run "./includes/configuration"

In [0]:
from pyspark.sql.functions import when, col

#### Get file name

In [0]:
source_object_name = dbutils.widgets.get('Source_Object_Name')
load_type = dbutils.widgets.get('Load_Type')
incre_folder_name = dbutils.widgets.get('Incre_Folder_Name')

#### Read bronze delta file

In [0]:
if load_type == 'full_load':
    df = spark.read.format('delta').load(f'{bronze_folder_path}/{source_object_name}')
elif load_type == 'incremental':
    df = spark.read.format('delta').load(f'{bronze_folder_path}/{source_object_name}/{incre_folder_name}')

In [0]:
display(df.limit(5))

S_ID,Tin_No,Company_Name,Date_of_Reg,SubCity,Town,Telephone,Fax,EMail,Business_License_No,Business_Type
1.0,Local-0004765279,ßììßëàßê¡ ßîáßëàßêïßêï ßèòßîìßï╡ Fiker General Trading,21-03-2019,Akaki Kality,Addis Ababa,2.51911E+11,2.51114E+11,betebebubi@yahoo.com,14/665/1910/2005,Computers and Related Equipments
2.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653400/2009,Chemicals and chemical products
3.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653407/2009,Laboratory materials and Equipment
4.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,03/213/01/3025/2005,Grade VI
5.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,AM/BD/HDR/03/213/6011912/2008,Building Materials


### Apply Business Rule

In [0]:
if source_object_name == "supplier":
    # Adding negotiation score and defect quality column based on business type in supplier
    df = df.withColumn('Negotiation_Score', 
                       when(col('Business_Type') == 'Printing Press', 0.9)
                       .when(col('Business_Type') == 'Stationery Materials', 0.8)
                       .when(col('Business_Type') == 'Software Development and Design', 0.95)
                       .otherwise(0.75)) \
            .withColumn('Defect_Quality',
                        when(col('Business_Type').isin('Detergents', 'Sanitary Items'), 'High') \
                        .when(col('Business_Type').isin('Building and Construction Materials', 
                                                        'Metal and Metal Products'), 'Medium') \
                        .otherwise('Low'))
            
elif source_object_name == "transportation":
    # Adding priority column based on modes in transportation
    df = df.withColumn('Priority',
                       when(col('MODES') == 'Truck', 1)
                       .when(col('MODES') == 'Ship', 2)
                       .when(col('MODES') == 'Airplane', 3)
                       .otherwise(0))
    
elif source_object_name == "product":
    # Adding price category column in product
    df = df.withColumn('Price_Category',
                       when(col('Price') < 50, 'Low Price')
                       .when((col('Price') > 50) & (col('Price') < 200), 'Medium Price')
                       .when(col('Price') > 200, 'High Price')
                       .otherwise('Unknown'))

elif source_object_name == 'purchaseorder':
    # Adding Total cost column in purchaseorder
    df = df.withColumn('Total_Cost', col('Order_quantities') * col('Costs'))

In [0]:
display(df.limit(5))

S_ID,Tin_No,Company_Name,Date_of_Reg,SubCity,Town,Telephone,Fax,EMail,Business_License_No,Business_Type,Negotiation_Score,Defect_Quality
1.0,Local-0004765279,ßììßëàßê¡ ßîáßëàßêïßêï ßèòßîìßï╡ Fiker General Trading,21-03-2019,Akaki Kality,Addis Ababa,2.51911E+11,2.51114E+11,betebebubi@yahoo.com,14/665/1910/2005,Computers and Related Equipments,0.75,Low
2.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653400/2009,Chemicals and chemical products,0.75,Low
3.0,Local-0052225274,ßîîßë╡ßê░ßèò ßèóßèòßï│ßê╡ßë╡ßê¬ßê╡ ßèâßêï/ßï¿ßë░/ßï¿ßîì/ßê¢ßêàßëáßê¡ Getson Industries Pvt L,17-03-2017,Kirkos,Addis Ababa,2.5191E+11,2.51116E+11,info@getson.et,MT/AA/14/706/653407/2009,Laboratory materials and Equipment,0.75,Low
4.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,03/213/01/3025/2005,Grade VI,0.75,Low
5.0,Local-0005051214,ßï│ßîìßê¢ßïè ßê¥ßê╡ßîïßèôßïì ßêÑßê½ ßë░ßëïßê½ßî¡ßìí ßê¢ßê╜ßèÉßê¬ ßè¬ßê½ßï¡ ßèÑßèô ßï¿ßîìßèòßëúßë│ ßèÑßëâßïÄßë╜ ßîàßê¥ßêï ßèòßîìßï╡,06-09-2019,Atsie Tewodros kifil,Bahir dar,911468033,918764449,dagmye63@gmail.com,AM/BD/HDR/03/213/6011912/2008,Building Materials,0.75,Low


#### Write transformed data to silver container in delta format

In [0]:
if load_type == 'full_load':
    df.write.format('delta').mode('overwrite').save(f'{silver_folder_path}/{source_object_name}')
    # Creating a Delta Table instance
    from delta.tables import DeltaTable
    dt = DeltaTable.forPath(spark, f'{silver_folder_path}/{source_object_name}')
    # Removing old history
    dt.vacuum(retentionHours=24)

elif load_type == 'incremental':
    df.write.format('delta').mode('append').save(f'{silver_folder_path}/{source_object_name}')